<a href="https://colab.research.google.com/github/zeliemarcais/Dashboard-of-Unique-Words---Shiny/blob/main/code/index/transport_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is to get the number of people that have access to public transportation in a city (less than 1km away from a public transportation stop), using open street maps data.
It is quite long to run, should be done with a VM (cloud computing).


In [ ]:
%pip install osmnx
%pip install folium matplotlib mapclassify
!pip install rasterstats
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 11.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import box
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import ee
import json
import geemap.core as geemap
from shapely.geometry import MultiPoint, Polygon, MultiPolygon
from random import sample

In [ ]:
#allow colab to access google drive for ghsl data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


LOAD THE DATA

In [ ]:
# Read the urban centers spatial dataset
uc = gpd.read_file('/content/gdrive/MyDrive/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')
uc = uc[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_NM', 'geometry']]

#subset from where last left off
uc = uc.iloc[1:13135]

In [ ]:
#WHEN USING A VM TO RUN CODE

#import ee
service_account = '654691552728-compute@developer.gserviceaccount.com	'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-zelie-marcais-096d46df8141.json')
ee.Initialize(credentials)

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-zeliemarcais')

Get the population data for each year

In [ ]:
#get population data for missing years
pop_2015 = ee.Image("JRC/GHSL/P2023A/GHS_POP/2015")
pop_2020 = ee.Image("JRC/GHSL/P2023A/GHS_POP/2020")
pop_2025 = ee.Image("JRC/GHSL/P2023A/GHS_POP/2025")

# Function to linearly interpolate/extrapolate population rasters
def interpolate_pop(year, pop_start, year_start, pop_end, year_end):
    annual_change = pop_end.subtract(pop_start).divide(ee.Number(year_end - year_start))
    years_since_start = ee.Number(year).subtract(year_start)
    interpolated_pop = pop_start.add(annual_change.multiply(years_since_start))
    return interpolated_pop

# Generate population rasters for missing years
years = list(range(2016, 2020)) + list(range(2021, 2025))
pop_images = {}

# Interpolate for 2016-2019 based on 2015-2020
for year in range(2016, 2020):
    pop_images[year] = interpolate_pop(year, pop_2015, 2015, pop_2020, 2020)

# Interpolate for 2021-2024 based on 2020-2025
for year in range(2021, 2025):
    pop_images[year] = interpolate_pop(year, pop_2020, 2020, pop_2025, 2025)

# Print out the generated images for verification
for year in years:
    print(f"Population raster for {year}: {pop_images[year].getInfo()}")

pop_2016 = pop_images[2016]
pop_2017 = pop_images[2017]
pop_2018 = pop_images[2018]
pop_2019 = pop_images[2019]

pop_2021 = pop_images[2021]
pop_2022 = pop_images[2022]
pop_2023 = pop_images[2023]
pop_2024 = pop_images[2024]


Population raster for 2016: {'type': 'Image', 'bands': [{'id': 'population_count', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [100, 0, -18041000, 0, -100, 9000000]}]}
Population raster for 2017: {'type': 'Image', 'bands': [{'id': 'population_count', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DA

DEFINE FUNCTION TO GET NETWORK OF STOPS

In [ ]:
def get_network(city_geometry, city_name, city_id, tags, year):

    #get all the features of transportation stops for a city by city shape (including attributes and geometries)
    network = ox.features.features_from_polygon(
        city_geometry,
        tags=tags)

    # if network is empty, the error will be processed outside the funcion
    gdf_network = gpd.GeoDataFrame.from_features(network, crs="EPSG:4326")
    gdf_network = gdf_network.to_crs("EPSG:4326")

    #convert all geometries of the transportation stops to points, using center for polygons (train statiions for ex)
    gdf_network['geometry'] = gdf_network['geometry'].apply(
        lambda geom: MultiPoint([geom.centroid]) if geom.is_empty else geom
    )

   # if not combined_geometry.empty:
    combined_geometry = gdf_network['geometry'].unary_union
    #else
     #   combined_geometry = gdf_network['geometry']

    city_data = {
        "geometry": [combined_geometry],
        "city": [city_name],
        "id": [city_id],
        "year": [year]
    }

    city_gdf = gpd.GeoDataFrame(city_data)
    return city_gdf


DEFINE FUNCTION TO GET POPULATION COVERAGE FROM GEE

In [ ]:
#function for population from GEE

def get_population(city_gdf, year, city_borders):

    #create buffer zones around each stop, resolving for overlap. buffer zone = 15 min walk = 1km
    #remove the previous geometry colum (the points of stops, not the polygon of the entire are covered by public transportation)
    buffer_distance = 1
    city_gdf['buffer'] = city_gdf['geometry'].buffer(buffer_distance / 111)
    #city_gdf = city_gdf.drop(columns='geometry')
    city_gdf = city_gdf[['city', 'id', 'buffer']]
    city_gdf = city_gdf.rename(columns={'buffer': 'geometry'})

    #clip buffers to be inside UC geom
    city_gdf_clipped = gpd.clip(city_gdf, city_borders)

    # Convert GeoPandas DataFrame to GeoJSON
    geojson_data = gpd.GeoSeries(city_gdf_clipped['geometry']).__geo_interface__

    # Extract properties from the GeoDataFrame: Before converting the GeoJSON features into an Earth Engine FeatureCollection,
    #properties such as city name and ID are extracted from the GeoDataFrame (properties_df) and added to each GeoJSON feature.
    #This ensures that relevant information is preserved and associated with each geographical feature during subsequent analysis.
    properties_df = city_gdf_clipped.drop(columns=['geometry'])

    # Add properties to each feature in the GeoJSON data
    for i, feature in enumerate(geojson_data['features']):
        feature['properties'] = properties_df.iloc[i].to_dict()

    # Create an Earth Engine FeatureCollection from the modified GeoJSON data
    uc_fc = ee.FeatureCollection(geojson_data['features'])

    if year == 2015:
      pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2015")
    elif year == 2020:
      pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2020")
    elif year == 2025:
      pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2025")
    else:
        if year in pop_images:
          pop = pop_images[year]
        else:
          raise ValueError(f"Population data for the year {year} is not available.")

    # Replace NoData pixels of population raster with the fill
    fill_value = 0
    pop_no_data_filled = pop.unmask(fill_value)
    population_clipped = pop_no_data_filled.clip(uc_fc)

    #get the sum of population in the area of the city covered by public transportation using reducer.sum
    uc_stats = population_clipped.reduceRegions(
        collection=uc_fc,
        reducer=ee.Reducer.sum(),
        scale=population_clipped.projection().nominalScale(),
        crs=population_clipped.projection().wkt())

    uc_pdf = ee.data.computeFeatures({
        'expression': uc_stats,
        'fileFormat': 'PANDAS_DATAFRAME'
    })

    uc_pdf = uc_pdf.drop(columns='geo')

    return uc_pdf


# RUN FUNCTIONS
Extract the number of people within the area of a city that is 1km away from a transportation stop

In [ ]:
tags = {'highway': ['bus_stop'],
      'station': ['subway', 'light_rail', 'monorail', 'funicular', 'tram', 'bus'],
      'railway': ['stop', 'station'],
      'trolleybus': ['yes'],
      'trolley_wire': ['yes']}


#empty data frame to store results
coverage_all = gpd.GeoDataFrame(columns=['city', 'id', 'sum'])
counter = 13000 #track what city we are at
year = 2024

ox.settings.log_file = True
ox.settings.logs_folder = '/content/'
ox.settings.overpass_settings = "[out:json][timeout:300][date:'" + str(year) + "-04-31T23:59:00Z']"
ox.settings.timeout = 300

for index, row in uc.iterrows():
    city_geometry = row["geometry"]
    city_name = row["UC_NM_MN"]
    city_id = row["ID_HDC_G0"]

    # Start time of the execution
   # start_time = time.time()

    #pass the osm function to the city
    try:
        X = get_network(city_geometry, city_name, city_id, tags, year)
        #X = ox.features.features_from_polygon(city_geometry, tags=tags)
    except Exception as e:
        print(f"Error processing osm {city_name}: {e}")
        continue

    # pass the population function to the city
    try:
        uc_pdf = get_population(X, year, city_geometry)
    except Exception as e:
        print(f"Error processing population {city_name}: {e}")
        continue

    #append city results to the final dataframe
    coverage_all = pd.concat([coverage_all, uc_pdf], ignore_index = True)

    counter += 1
    #keep track of progress: export and print group results
    if counter % 100 == 0:
        coverage_all.to_csv(f'/content/gdrive/MyDrive/exported_last_{counter}_cities_{year}.csv', index=True)
        print(f"Exported {counter} cities with networks.")


INFO:OSMnx:Projected GeoDataFrame to 'EPSG:32654 / WGS 84 / UTM zone 54N'
INFO:OSMnx:Projected GeoDataFrame to 'EPSG:4326 / WGS 84'
INFO:OSMnx:Requesting data from API in 1 request(s)
INFO:OSMnx:Resolved 'overpass-api.de' to '65.109.112.52'
INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%2735.399740+140.314344+35.399740+140.291691+35.408481+140.278524+35.417221+140.265358+35.434704+140.273019+35.452189+140.292025+35.452189+140.326018+35.417221+140.333327+35.399740+140.314344%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%2735.399740+140.314344+35.399740+140.291691+35.408481+140.278524+35.417221+140.265358+35.434704+140.273019+35.452189+140.292025+35.452189+140.326018+35.417221+140.333327+35.399740+140.314344%27%29%3

Error processing osm Sone: No matching features. Check query location, tags, and log.


INFO:OSMnx:Projected GeoDataFrame to 'EPSG:32651 / WGS 84 / UTM zone 51N'
INFO:OSMnx:Projected GeoDataFrame to 'EPSG:4326 / WGS 84'
INFO:OSMnx:Requesting data from API in 1 request(s)
INFO:OSMnx:Resolved 'overpass-api.de' to '65.109.112.52'
INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%277.172866+124.215328+7.213427+124.212150+7.221539+124.213525+7.237764+124.226306+7.237764+124.256387+7.221539+124.263659+7.205314+124.260911+7.180978+124.246774+7.172866+124.235380+7.172866+124.215328%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%277.172866+124.215328+7.213427+124.212150+7.221539+124.213525+7.237764+124.226306+7.237764+124.256387+7.221539+124.263659+7.205314+124.260911+7.180978+124.246774+7.172866+124.235380+7.172

Error processing osm Cotabato: No matching features. Check query location, tags, and log.


INFO:OSMnx:Projected GeoDataFrame to 'EPSG:4326 / WGS 84'
INFO:OSMnx:Requesting data from API in 1 request(s)
INFO:OSMnx:Resolved 'overpass-api.de' to '65.109.112.52'
INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-8.236044+124.508346+-8.236044+124.488263+-8.227924+124.486688+-8.219804+124.495155+-8.211684+124.503625+-8.203565+124.512095+-8.203565+124.542219+-8.203565+124.562302+-8.203565+124.572343+-8.219804+124.575488+-8.227924+124.556979+-8.236044+124.508346%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-8.236044+124.508346+-8.236044+124.488263+-8.227924+124.486688+-8.219804+124.495155+-8.211684+124.503625+-8.203565+124.512095+-8.203565+124.542219+-8.203565+124.562302+-8.203565+124.572343+-8.219804+124.5754

...

INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-37.826170+175.280290+-37.781953+175.214745+-37.773111+175.213200+-37.737746+175.218603+-37.720066+175.238638+-37.720066+175.273261+-37.728906+175.286328+-37.764269+175.315567+-37.790796+175.331774+-37.808482+175.334902+-37.826170+175.303392+-37.826170+175.280290%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-37.826170+175.280290+-37.781953+175.214745+-37.773111+175.213200+-37.737746+175.218603+-37.720066+175.238638+-37.720066+175.273261+-37.728906+175.286328+-37.764269+175.315567+-37.790796+175.331774+-37.808482+175.334902+-37.826170+175.303392+-37.826170+175.280290%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28relation%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-37.

Error processing osm Kokopo: No matching features. Check query location, tags, and log.


INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-37.084632+174.932933+-37.049389+174.859236+-36.934892+174.649095+-36.908479+174.611328+-36.864464+174.582873+-36.855663+174.581779+-36.829260+174.589976+-36.820460+174.600352+-36.811660+174.610730+-36.811660+174.645093+-36.838060+174.854567+-36.846861+174.878588+-36.855663+174.902617+-36.899675+174.965554+-37.075821+174.989114+-37.084632+174.955893+-37.084632+174.932933%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-37.084632+174.932933+-37.049389+174.859236+-36.934892+174.649095+-36.908479+174.611328+-36.864464+174.582873+-36.855663+174.581779+-36.829260+174.589976+-36.820460+174.600352+-36.811660+174.610730+-36.811660+174.645093+-36.838060+174.854567+-36.846861

Error processing osm Buin: No matching features. Check query location, tags, and log.


INFO:OSMnx:Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
INFO:OSMnx:Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A300%5D%5Bdate%3A%272024-04-31T23%3A59%3A00Z%27%5D%3B%28%28node%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-9.454775+160.001352+-9.454775+159.951040+-9.446645+159.938646+-9.438515+159.926254+-9.430385+159.913864+-9.422255+159.911539+-9.422255+159.961847+-9.422255+159.992033+-9.422255+160.002094+-9.438515+160.006750+-9.454775+160.001352%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28way%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-9.454775+160.001352+-9.454775+159.951040+-9.446645+159.938646+-9.438515+159.926254+-9.430385+159.913864+-9.422255+159.911539+-9.422255+159.961847+-9.422255+159.992033+-9.422255+160.002094+-9.438515+160.006750+-9.454775+160.001352%27%29%3B%28._%3B%3E%3B%29%3B%29%3B%28relation%5B%27highway%27%3D%27bus_stop%27%5D%28poly%3A%27-9.454775+160.001352+-9.454775+159.951040+-9.446645+159.938646+-9.4385

In [ ]:
coverage_all.to_csv(f'/content/gdrive/MyDrive/exported_{counter}_cities_{year}.csv')

VISUALIZE LAYERS - Check buffer zones etc

In [ ]:
#import geemap

# Convert GeoPandas DataFrame to GeoJSON
#geojson_data = gpd.GeoSeries(uc_subset['geometry']).__geo_interface__

# Extract properties from the GeoDataFrame
#properties_df = uc_subset.drop(columns=['geometry'])

# Add properties to each feature in the GeoJSON data
#for i, feature in enumerate(geojson_data['features']):
 #   feature['properties'] = properties_df.iloc[i].to_dict()

#Create an Earth Engine FeatureCollection from the modified GeoJSON data
#subset_2 = ee.FeatureCollection(geojson_data['features'])



# Create a map object
#Map = geemap.Map()

# Center the map on the subset_2 feature collection
#Map.centerObject(subset_2)

# Add Urban Centers layer to the map
#Map.addLayer(subset_2, {}, 'Urban Centers')

# Convert GeoDataFrame to Earth Engine FeatureCollection
# First, ensure that the GeoDataFrame has a defined CRS
#X.crs = 'EPSG:4326'  # Assuming WGS84, change this if your data uses a different CRS
#buffer = X.drop(columns='geometry')
#buffer = buffer.rename(columns={'buffer': 'geometry'})
#buffer = gpd.clip(buffer, uc_subset)


#buffer.crs= 'EPSG:4326'
# Convert 'buffer' column to Earth Engine FeatureCollection
#buffer_fc = geemap.geopandas_to_ee(buffer)

# Add Buffer layer to the map
#Map.addLayer(buffer_fc, {}, 'Buffer')

# Display the map
#Map

In [ ]:
#map with population density background

# Convert GeoPandas DataFrame to GeoJSON for urban centers
geojson_data = gpd.GeoSeries(uc['geometry']).__geo_interface__

# Extract properties from the GeoDataFrame
properties_df = uc.drop(columns=['geometry'])

# Add properties to each feature in the GeoJSON data
for i, feature in enumerate(geojson_data['features']):
    feature['properties'] = properties_df.iloc[i].to_dict()

# Create an Earth Engine FeatureCollection from the modified GeoJSON data
subset_2 = ee.FeatureCollection(geojson_data['features'])

# Convert GeoDataFrame to Earth Engine FeatureCollection
# First, ensure that the GeoDataFrame has a defined CRS
X.crs = 'EPSG:4326'  # Assuming WGS84, change this if your data uses a different CRS
buffer = X.drop(columns='geometry')
buffer = buffer.rename(columns={'buffer': 'geometry'})
buffer = gpd.clip(buffer, uc)

buffer.crs = 'EPSG:4326'
# Convert 'buffer' column to Earth Engine FeatureCollection
buffer_fc = geemap.geopandas_to_ee(buffer)

# Load population density data (replace 'pop_2022' with your actual asset or image)
pop_2016 = pop_images[2016]

# Create a map object
Map = geemap.Map()

# Add the population density layer with a green palette
Map.addLayer(pop_2016, {'min': 0, 'max': 100, 'palette': ['white', 'green']}, 'Population Density')

# Style for city boundaries (blue outlines)
style_uc = {
    'color': 'blue',
    'width': 1
}

# Style for buffer zones (red outlines)
style_buffer = {
    'color': 'red',
    'width': 1
}

# Convert urban centers to GeoJSON dictionary
uc_geojson = uc.to_crs('EPSG:4326').__geo_interface__

# Convert buffer zones to GeoJSON dictionary
buffer_geojson = buffer.to_crs('EPSG:4326').__geo_interface__

# Add the urban centers layer with blue outlines
Map.add_geojson(uc_geojson, layer_name='Urban Centers', style=style_uc)

# Add the buffer zones layer with red outlines
Map.add_geojson(buffer_geojson, layer_name='Buffer Zones', style=style_buffer)

# Center the map on the urban centers
Map.centerObject(subset_2, 10)

# Display the map
Map

